In [1]:
import os
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import glob
import cv2
import concurrent.futures
import PIL.Image

In [2]:
root_dir = os.path.join(os.getcwd(), 'images')

In [3]:

# Just normalization for validation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
train_set = torchvision.datasets.ImageFolder(root=os.path.join(root_dir, 'train'), transform=transform)
test_set = torchvision.datasets.ImageFolder(root=os.path.join(root_dir, 'test'), transform=transform)
val_set = torchvision.datasets.ImageFolder(root=os.path.join(root_dir, 'val'), transform=transform)

In [5]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=32, shuffle=False, num_workers=4)

In [6]:
dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': len(train_set), 'val': len(val_set)}

In [7]:
model = torchvision.models.vgg16(pretrained=True)

In [8]:
model.classifier[6] = torch.nn.Linear(4096, 8)

In [9]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [10]:
model = model.to(device)

In [11]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [17]:
for epoch in range(5):
    print('Epoch {}/{}'.format(epoch, 5 - 1))

    for input , label in train_loader:
        input = input.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        print("im here1")
        output = model(input)
        print("im here2")
        loss = criterion(output, label)
        print("im here3")
        loss.backward()
        print("im here4")
        optimizer.step()
        print("im here5")
    print('Loss: {:.4f}'.format(loss.item()))
    print('-' * 10)

Epoch 0/4
im here1


Process Process-17:
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f12e10f5f70>
Traceback (most recent call last):
  File "/home/hassen/pycharm-download/TER/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1327, in __del__
  File "/home/hassen/pycharm-download/TER/lib/python3.8/site-packages/torch/utils/data/_utils/signal_handling.py", line 66, in handler
RuntimeError: DataLoader worker (pid 1043810) exited unexpectedly with exit code 1. Details are lost due to multiprocessing. Rerunning with num_workers=0 may give better error trace.
Exception ignored in: <function WeakValueDictionary.__init__.<locals>.remove at 0x7f12e11e0e50>
Traceback (most recent call last):
  File "/usr/lib/python3.8/weakref.py", line 103, in remove
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
def accuracy(outputs , labels ):
    _, preds = torch.max(outputs, 1)
    return torch.sum(preds == labels).item() / len(preds)

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for input , label in test_loader:
        input = input.to(device)
        label = label.to(device)
        outputs = model(input)
        correct += accuracy(outputs, label)
        total += 1
    print('Accuracy of the network on the validation images: %d %%' % (100 * correct / total))